In [1]:
!pip install langchain-chroma langchain pandas pypdf openai langchain-ollama

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_ollama import OllamaLLM
# deepseek-r1:1.5b R1 모델 로드
deepseek = OllamaLLM(model="deepseek-r1:1.5b")

In [3]:
from langchain.document_loaders import PyPDFLoader
docs = [] # 추출된 문서를 저장할 리스트를 초기화

loader = PyPDFLoader("d:/data/안구건조증.pdf") # "d:/data/안구건조증.pdf" 경로의 PDF 파일을 PyPDFLoader로 로드

# PDF에서 텍스트 데이터를 추출하여 리스트 형태로 반환
text_content = loader.load()

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma # 벡터 데이터를 저장하고 검색하는 벡터 데이터베이스 (ChromaDB)
from langchain.text_splitter import CharacterTextSplitter

# 문서 분할 (텍스트를 작은 단위로 나누기)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

# 문서를 CharacterTextSplitter를 사용하여 분할
splits = text_splitter.split_documents(text_content)
# 문서를 CharacterTextSplitter를 사용하여 분할
texts = [doc.page_content for doc in splits]

# 문서를 CharacterTextSplitter를 사용하여 분할
embeddings = OpenAIEmbeddings()

# 변환된 벡터를 Chroma 벡터 데이터베이스에 저장
vectorstore = Chroma(persist_directory='d:/data/db_faiss_complete', embedding_function=embeddings)

In [5]:
from langchain_core.prompts import PromptTemplate
custom_prompt_template = PromptTemplate(
    template = """당신은 안구건조증에 대한 정보를 제공하는 건강 지킴이 챗봇입니다.

{context}

제공된 소스 데이터만 사용하여 고객의 질문에 답하세요.
모르겠다면 "잘 모르겠습니다."라고 답하세요.
친절하고 예의 바르며 전문적인 어조를 사용하세요.
답변은 간결하게 유지하세요.

질문: {question}

답변:
""",
input_variables=["context"]
)
chain_type_kwargs = {"prompt": custom_prompt_template}

In [6]:
query = "습도가 안구건조증에 미치는 영향은?"
retriever = vectorstore.similarity_search(query,k=8) # vectorstore(ChromaDB 벡터 저장소)에서 검색 수행

In [7]:
from langchain.chains import RetrievalQA

# RetrievalQA.from_chain_type()을 사용하여 QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=deepseek,
    chain_type="stuff",  # 기본적으로 retrieved 문서를 합쳐서 처리
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs=chain_type_kwargs   # 
)

# 사용자의 질문을 입력하여 검색 및 응답 생성
result = qa_chain.invoke({"query": query})

result["result"]

"<think>\nOkay, so I need to figure out the effects of high potassium on anabolic syndrome. Hmm, let me start by recalling what anabolic syndrome is all about. It's a sleep disorder characterized by excessive sleep eating (ESE), which is when people eat while sleeping more than usual.\n\nNow, potassium levels play a role in many bodily processes, including electrolyte balance, muscle function, and nerve activity. High potassium can affect things like blood pressure, heart rate, and the nervous system. I remember that high potassium can lead to edema, fluid buildup, which is related to anabolic syndrome since it often involves fluid issues.\n\nBut wait, how does that specifically affect anabolic syndrome? Anabolic syndrome's primary symptoms are ESE, which as I said requires a lot of electrolytes like K+ ions. So, if someone has high potassium due to anabolic syndrome, maybe there could be some imbalance in their electrolyte balance.\n\nIn terms of blood pressure: elevated potassium lev

In [8]:
query = "인공눈물은 자주 사용해도 되는거야?"
result = qa_chain.invoke({"query": query})
result["result"]

"<think>\nAlright, I need to answer the question about whether people frequently use artificial eyes. First, I'll think about different ways people might use them—like gaming or training their eyes. Then I'll consider if they're used in everyday tasks like driving or using screens. I should also address cases where they aren't used at all and mention that it's subjective since some people find it helpful while others don't.\n\nI'll structure the answer to include both positive uses, like gaming and training, and negative points, such as not being useful for daily activities. This way, it covers a broad range of perspectives and provides clear information.\n</think>\n\n인공눈물은some People, like those who play games or train their eyes, might frequently use them. However, others may find little use for them in everyday tasks like driving or using screens. It's subjective, but many people perceive artificial eyes as helpful in certain situations."